In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
#first, we need to establish a connection to our database

conn = sqlite3.connect('ufc_database.db') 
cursor = conn.cursor()

In [3]:
#next, we'll use pandas to turn our current fighters data into a pandas dataframe
fighters_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM joined_current_fighters
                               ''', conn)

current_fighters = pd.DataFrame(fighters_query)

In [4]:
#let's commit and close the database for now
conn.commit()
conn.close()

In [5]:
#now, our table had included several duplicates of Mike Davis, but not all of them are created equal
#let's take a look:

dups = current_fighters[current_fighters['full_nm'].str.contains('Mike Davis')]
print(dups)

       full_nm   nickname     ht        wt  reach    stance   w  l  d   belt  \
80  Mike Davis                --        --     --             2  0  0  FALSE   
81  Mike Davis  Beast Boy  6' 0"  155 lbs.  72.0"  Orthodox  10  2  0  FALSE   
83  Mike Davis                --        --     --             2  0  0  FALSE   
84  Mike Davis  Beast Boy  6' 0"  155 lbs.  72.0"  Orthodox  10  2  0  FALSE   

    ... age_in_2023 sig_str_pm str_acc_percentage str_abs_pm  \
80  ...          27       0.00                  0       0.00   
81  ...          27       0.00                  0       0.00   
83  ...          27       5.83                 52       6.20   
84  ...          27       5.83                 52       6.20   

   str_def_percentage td_avg_15m td_acc_percentage td_def_percentage  \
80                  0       0.00                 0                 0   
81                  0       0.00                 0                 0   
83                 56       3.04                53            

In [6]:
#it looks like row 84 contains the most information, so we'll delete the rest

current_fighters = current_fighters.drop(labels=[80,81,83], axis=0)

In [7]:
# next, we want to convert the ht column from the feet'inches" format to total inches

# first, we'll ensure that it's a string type
current_fighters['ht'] = current_fighters['ht'].astype(str)

# next, create the function
def convert_height(height):
    if height == "--":
        return None
    else:
        feet_inches = height.split("' ")
        if len(feet_inches) == 2:
            feet, inches = feet_inches
            total_inches = int(feet) * 12 + int(inches.strip('"'))
            return total_inches
        else:
            return None

# apply the function to the 'ht' column
current_fighters['ht'] = current_fighters['ht'].apply(convert_height)

In [8]:
#our weight and reach columna have the object dtype, so we'll need to use rstrip and to_numeric so we can analyze them later

current_fighters['wt'] = current_fighters['wt'].str.rstrip(' lbs.')
current_fighters['wt'] = pd.to_numeric(current_fighters['wt'])

current_fighters['reach'] = current_fighters['reach'].str.rstrip('""')
current_fighters['reach'] = pd.to_numeric(current_fighters['reach'])

In [9]:
current_fighters.head()

,full_nm,nickname,ht,wt,reach,stance,w,l,d,belt,...,age_in_2023,sig_str_pm,str_acc_percentage,str_abs_pm,str_def_percentage,td_avg_15m,td_acc_percentage,td_def_percentage,sub_avg_15m,weight_class
0,Shamil Abdurakhimov,Abrek,75,235,76.0,Orthodox,20,8,0,FALSE,...,29,2.41,44,3.02,55,1.01,23,45,0.1,Heavyweight
1,Mariya Agapova,Demonslayer,66,125,68.0,Southpaw,10,4,0,FALSE,...,27,4.67,55,3.82,52,0.59,66,47,0.9,Women's Flyweight
2,Kevin Aguilar,Angel of Death,67,155,73.0,Orthodox,17,5,0,FALSE,...,27,3.96,40,4.81,52,0.16,16,78,0.0,Featherweight
3,Amir Albazi,The Prince,65,125,68.0,Orthodox,17,1,0,FALSE,...,31,2.80,39,3.07,63,1.71,33,40,0.7,Flyweight
4,Irene Aldana,,69,135,68.0,Orthodox,14,7,0,FALSE,...,31,4.86,39,5.71,59,0.17,50,75,0.3,Catch Weight


In [10]:
#a lot of our columns are objects, but we want to change them to num types

import re
num_cols = ['w', 'l', 'd','age_in_2023', 'sig_str_pm', 'str_acc_percentage',
       'str_abs_pm', 'str_def_percentage', 'td_avg_15m', 'td_acc_percentage',
       'td_def_percentage', 'sub_avg_15m']

for col in num_cols:
    if current_fighters[col].dtype == 'object':
        current_fighters[col] = current_fighters[col].apply(lambda x: re.sub(r'^\s+', '', str(x)))
        current_fighters[col] = pd.to_numeric(current_fighters[col])

#let's just change the rest of the columns to their best forms if possible
current_fighters = current_fighters.convert_dtypes()

In [11]:
#we want to change belt to bool
#current_fighters['belt'] = current_fighters['belt'].map({'TRUE': True, 'FALSE': False})

In [12]:
#now we want to pre-process the data a little more for a machine-learning model

In [13]:
#first, let's create a new column 'win_percentage' that we can do some exploratory linear regression on

total = current_fighters['w'] + current_fighters['l'] + current_fighters['d']
wins = current_fighters['w'] / total

current_fighters['win_percentage'] = wins

In [14]:
#next, let's change the other percentage columns to actual percents

per_cols = ['str_acc_percentage', 'str_def_percentage','td_acc_percentage', 'td_def_percentage']

for col in per_cols:
    current_fighters[col] = current_fighters[col] / 100
    
current_fighters.head()

,full_nm,nickname,ht,wt,reach,stance,w,l,d,belt,...,sig_str_pm,str_acc_percentage,str_abs_pm,str_def_percentage,td_avg_15m,td_acc_percentage,td_def_percentage,sub_avg_15m,weight_class,win_percentage
0,Shamil Abdurakhimov,Abrek,75,235,76,Orthodox,20,8,0,FALSE,...,2.41,0.44,3.02,0.55,1.01,0.23,0.45,0.1,Heavyweight,0.714286
1,Mariya Agapova,Demonslayer,66,125,68,Southpaw,10,4,0,FALSE,...,4.67,0.55,3.82,0.52,0.59,0.66,0.47,0.9,Women's Flyweight,0.714286
2,Kevin Aguilar,Angel of Death,67,155,73,Orthodox,17,5,0,FALSE,...,3.96,0.4,4.81,0.52,0.16,0.16,0.78,0.0,Featherweight,0.772727
3,Amir Albazi,The Prince,65,125,68,Orthodox,17,1,0,FALSE,...,2.8,0.39,3.07,0.63,1.71,0.33,0.4,0.7,Flyweight,0.944444
4,Irene Aldana,,69,135,68,Orthodox,14,7,0,FALSE,...,4.86,0.39,5.71,0.59,0.17,0.5,0.75,0.3,Catch Weight,0.666667


In [15]:
#finally, we want to encode our categorical values 

In [16]:
def replace_boolean_strings(value):
    if value == "FALSE":
        return False
    elif value == "TRUE":
        return True
    else:
        return value
current_fighters['belt'] = current_fighters['belt'].apply(replace_boolean_strings)
current_fighters['belt'] = current_fighters['belt'].astype(int)

In [17]:
#one-hot encode and drop our stance + weight class columns
current_fighters_pp = pd.get_dummies(current_fighters['stance'])

current_fighters = pd.concat([current_fighters, current_fighters_pp], axis=1)
current_fighters = current_fighters.drop('stance', axis=1)

In [18]:
current_fighters_pp = pd.get_dummies(current_fighters['weight_class'])

current_fighters = pd.concat([current_fighters, current_fighters_pp], axis=1)
current_fighters = current_fighters.drop('weight_class', axis=1)


In [19]:
current_fighters = current_fighters.drop('', axis=1)

In [20]:
#yay! now we'll save our pre-processed data into a new csv file to use for our exploration

current_fighters.to_csv('current_fighters_processed.csv')

In [21]:
#now, we'll process our other data so we have it available for exploration

conn = sqlite3.connect('ufc_database.db') 
cursor = conn.cursor()

historical_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM joined_historical_fight_stats
                               ''', conn)

historic_fights = pd.DataFrame(historical_query)

joined_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM joined_current_fighters_history
                               ''', conn)

fighters_history = pd.DataFrame(joined_query)

#and we'll commit and close
conn.commit()
conn.close()

In [22]:
#let's use python to convert all the object types to the most sensible dtype
historic_fights = historic_fights.convert_dtypes()

print(historic_fights.dtypes)

w_l               string
weight_class      string
method            string
round             string
time              string
event             string
date              string
location          string
fighter1          string
fighter2          string
kd_f1              Int64
kd_f2              Int64
str_f1             Int64
str_f2             Int64
td_f1              Int64
td_f2              Int64
sub_f1             Int64
sub_f2             Int64
ht_f1             string
ht_f2             string
wt_f1             string
wt_f2             string
reach_f1          string
reach_f2          string
stance_f1         string
stance_f2         string
w_f1              string
w_f2              string
l_f1              string
l_f2              string
d_f1              string
d_f2              string
belt_f1           string
belt_f2           string
country_f1        string
country_f2        string
age_in_2023_f1    string
age_in_2023_f2    string
active_f1         string
active_f2         string


In [23]:
def convert_height(height):
    if height is None or height == "--":
        return None
    try:
        feet, inches = height.split("'")
        feet = int(feet.strip())
        inches = int(inches.strip('"'))
        total_inches = feet * 12 + inches
        return total_inches
    except ValueError:
        return None

In [24]:
pd.set_option('display.max_columns', None)
fighters_history['ht_f1']

0       5' 11"
1        5' 4"
2        6' 0"
3        5' 8"
4       5' 10"
         ...  
2744     6' 0"
2745     5' 8"
2746     5' 5"
2747    5' 10"
2748     5' 6"
Name: ht_f1, Length: 2749, dtype: object

In [25]:
num_cols = ['round','ht_f1', 'ht_f2','wt_f1', 'wt_f2', 'reach_f1', 'reach_f2', 'w_f1', 'w_f2',
            'l_f1', 'l_f2', 'd_f1', 'd_f2',  'age_in_2023_f1', 'age_in_2023_f2']
ht_cols = ['ht_f1', 'ht_f2']

wt_cols = ['wt_f1', 'wt_f2']

rch_cols = ['reach_f1', 'reach_f2']

for col in ht_cols:
    fighters_history[col] = fighters_history[col].apply(convert_height)

for col in wt_cols:
    fighters_history[col] = fighters_history[col].astype(str)
    fighters_history[col].replace("--", None, inplace=True)
    fighters_history[col] = fighters_history[col].str.rstrip(" lbs.")
    
    historic_fights[col] = historic_fights[col].astype(str)
    historic_fights[col].replace("--", None, inplace=True)
    historic_fights[col] = historic_fights[col].str.rstrip(" lbs.")
    
for col in rch_cols:
    fighters_history[col] = fighters_history[col].str.rstrip('.0"')
    
for col in num_cols:
    historic_fights[col] = historic_fights[col].astype(str)
    historic_fights[col].replace("--", None, inplace=True)
    historic_fights[col].replace("#N/A", None, inplace=True)
    
    fighters_history[col] = fighters_history[col].astype(str)
    fighters_history[col].replace("--", None, inplace=True)
    fighters_history[col].replace("#N/A", None, inplace=True)

for col in num_cols:
    historic_fights[col] = pd.to_numeric(historic_fights[col], errors='coerce')
    
    fighters_history[col] = pd.to_numeric(fighters_history[col], errors='coerce')

In [26]:
print(fighters_history.dtypes)

w_l                object
weight_class       object
method             object
round               int64
time               object
event              object
date               object
location           object
fighter1           object
fighter2           object
kd_f1               int64
kd_f2               int64
str_f1              int64
str_f2              int64
td_f1               int64
td_f2               int64
sub_f1              int64
sub_f2              int64
ht_f1             float64
ht_f2             float64
wt_f1             float64
wt_f2             float64
reach_f1          float64
reach_f2          float64
stance_f1          object
stance_f2          object
w_f1                int64
w_f2                int64
l_f1                int64
l_f2                int64
d_f1                int64
d_f2                int64
belt_f1            object
belt_f2            object
country_f1         object
country_f2         object
age_in_2023_f1    float64
age_in_2023_f2    float64
active_f1   

In [27]:
str_cols = ['w_l', 'weight_class', 'method','event','location',
 'fighter1', 'fighter2','stance_f1', 'stance_f2','country_f1', 'country_f2']

for col in str_cols:
    historic_fights[col] = historic_fights[col].astype(str)
    historic_fights[col].replace("--", None, inplace=True)
    
    fighters_history[col] = historic_fights[col].astype(str)
    fighters_history[col].replace("--", None, inplace=True)

In [28]:
#get rid of the Mike Davis duplicates in historic fights + current fighters fight history
historic_fights = historic_fights.drop(labels=[219,1095,1656], axis=0) 

fighters_history = fighters_history.drop(labels=[179, 895, 1299], axis=0) 

In [29]:
#we'll encode our belt column in each df

belt_cols = ['belt_f1', 'belt_f2']

for col in belt_cols:

    historic_fights[col] = historic_fights[col].apply(replace_boolean_strings)
    historic_fights[col] = historic_fights[col].astype(int)

    fighters_history[col] = fighters_history[col].apply(replace_boolean_strings)
    fighters_history[col] = fighters_history[col].astype(int)

In [30]:
#now, we want one-hot encode our stance, weight_class, and method columns for both dfs

pp_cols = ['weight_class', 'method', 'stance_f1', 'stance_f2']

for col in pp_cols:
    historic_fights_pp = pd.get_dummies(historic_fights[col])
    historic_fights = pd.concat([historic_fights, historic_fights_pp], axis=1)
    historic_fights = historic_fights.drop(col, axis=1)


In [31]:
pp_cols = ['weight_class', 'method', 'stance_f1', 'stance_f2']

for col in pp_cols:
    fighters_history_pp = pd.get_dummies(fighters_history[col])
    fighters_history = pd.concat([fighters_history, fighters_history_pp], axis=1)
    fighters_history = fighters_history.drop(col, axis=1)

In [32]:
#now we have way too many encoded columns for methods. We're going to simplify these into four catagories instead
kos = ['KO/TKO','KO/TKO: Elbow','KO/TKO: Elbows','KO/TKO: Flying Knee','KO/TKO: Kick','KO/TKO: Kicks','KO/TKO: Knee','KO/TKO: Knees','KO/TKO: Punch','KO/TKO: Punches','KO/TKO: Slam','KO/TKO: Spinning Back Elbow','KO/TKO: Spinning Back Fist','KO/TKO: Spinning Back Kick']
decs = ['M-DEC','S-DEC','U-DEC']
ot = ['Overturned','Overturned: Arm Triangle',"Overturned: D'Arce Choke",'Overturned: Guillotine Choke','Overturned: Kick','Overturned: Knee','Overturned: Punch','Overturned: Punches','Overturned: Rear Naked Choke']
subs = ['SUB: Anaconda Choke','SUB: Ankle Lock','SUB: Arm Triangle','SUB: Armbar','SUB: Bulldog Choke','SUB: Calf Slicer',"SUB: D'Arce Choke",'SUB: Ezekiel Choke','SUB: Forearm Choke','SUB: Guillotine Choke','SUB: Heel Hook','SUB: Injury','SUB: Inverted Triangle','SUB: Keylock','SUB: Kimura','SUB: Kneebar','SUB: Neck Crank','SUB: North-South Choke','SUB: Omoplata','SUB: Peruvian Necktie','SUB: Rear Naked Choke','SUB: Scarf Hold','SUB: Schultz Front Headlock','SUB: Straight Armbar','SUB: Suloev Stretch','SUB: Triangle Armbar','SUB: Triangle Choke','SUB: Twister','SUB: Von Flue Choke']

#we'll combine each method into one of the four catagories for each dataframe and delete the extraneous columns

fighters_history['ko'] = fighters_history[kos].any(axis=1).astype(int)
fighters_history['dec'] = fighters_history[decs].any(axis=1).astype(int)
fighters_history['overturned'] = fighters_history[ot].any(axis=1).astype(int)
fighters_history['sub'] = fighters_history[subs].any(axis=1).astype(int)

fighters_history = fighters_history.drop(kos, axis=1)
fighters_history = fighters_history.drop(decs, axis=1)
fighters_history = fighters_history.drop(ot, axis=1)
fighters_history = fighters_history.drop(subs, axis=1)

historic_fights['ko'] = historic_fights[kos].any(axis=1).astype(int)
historic_fights['dec'] = historic_fights[decs].any(axis=1).astype(int)
historic_fights['overturned'] = historic_fights[ot].any(axis=1).astype(int)
historic_fights['sub'] = historic_fights[subs].any(axis=1).astype(int)

historic_fights = historic_fights.drop(kos, axis=1)
historic_fights = historic_fights.drop(decs, axis=1)
historic_fights = historic_fights.drop(ot, axis=1)
historic_fights = historic_fights.drop(subs, axis=1)

In [33]:
#let's also create a new column for each fighter's win percentage so we have that info handy for exploration

fighters_history['win_percentage_f1'] = fighters_history['w_f1'] / (fighters_history['w_f1'] + fighters_history['l_f1'] + fighters_history['d_f1'])
fighters_history['win_percentage_f2'] = fighters_history['w_f2'] / (fighters_history['w_f2'] + fighters_history['l_f2'] + fighters_history['d_f2'])


In [34]:
current_fighters.head()

,full_nm,nickname,ht,wt,reach,w,l,d,belt,country,age_in_2023,sig_str_pm,str_acc_percentage,str_abs_pm,str_def_percentage,td_avg_15m,td_acc_percentage,td_def_percentage,sub_avg_15m,win_percentage,Orthodox,Southpaw,Switch,Bantamweight,Catch Weight,Featherweight,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight
0,Shamil Abdurakhimov,Abrek,75,235,76,20,8,0,0,RUS,29,2.41,0.44,3.02,0.55,1.01,0.23,0.45,0.1,0.714286,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Mariya Agapova,Demonslayer,66,125,68,10,4,0,0,KAZ,27,4.67,0.55,3.82,0.52,0.59,0.66,0.47,0.9,0.714286,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Kevin Aguilar,Angel of Death,67,155,73,17,5,0,0,USA,27,3.96,0.4,4.81,0.52,0.16,0.16,0.78,0.0,0.772727,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Amir Albazi,The Prince,65,125,68,17,1,0,0,RUS,31,2.8,0.39,3.07,0.63,1.71,0.33,0.4,0.7,0.944444,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,Irene Aldana,,69,135,68,14,7,0,0,MEX,31,4.86,0.39,5.71,0.59,0.17,0.5,0.75,0.3,0.666667,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [35]:
#and we'll save everything to new csv files to call in our model

historic_fights.to_csv('historic_fights.csv')
fighters_history.to_csv('fighters_history.csv')

In [36]:
current_fighters2 = pd.DataFrame(current_fighters[['full_nm','sig_str_pm','str_acc_percentage', 'str_abs_pm', 'str_def_percentage',
                                    'td_avg_15m', 'td_acc_percentage', 'td_def_percentage', 'sub_avg_15m']])

In [37]:
#next we'll merge the two dataframes so we have additional stats for fighter1 and fighter2 for each historic fight

#first, we'll merge the current_fighters info into fighters_history where fighter1 and fighter2 matches full_nm
merged_fighters = pd.merge(fighters_history, current_fighters2, left_on='fighter1', right_on='full_nm', how='left', suffixes=('', '_f1'))

merged_fighters = pd.merge(merged_fighters, current_fighters2, left_on='fighter2', right_on='full_nm', how='left', suffixes=('', '_f2'))

#then, we'll drop the unneccessary columns from the merge
merged_fighters = merged_fighters.drop(columns=['full_nm','CNC', 'DQ', 'Other', '','', 'full_nm_f2'])

In [38]:
#a few of our columns have the same name, so we need to differentiate them
merged_fighters.columns.values[49:52] += '_f1'

merged_fighters.columns.values[52:55] += '_f2'

In [39]:
merged_fighters.columns

Index(['w_l', 'round', 'time', 'event', 'date', 'location', 'fighter1',
       'fighter2', 'kd_f1', 'kd_f2', 'str_f1', 'str_f2', 'td_f1', 'td_f2',
       'sub_f1', 'sub_f2', 'ht_f1', 'ht_f2', 'wt_f1', 'wt_f2', 'reach_f1',
       'reach_f2', 'w_f1', 'w_f2', 'l_f1', 'l_f2', 'd_f1', 'd_f2', 'belt_f1',
       'belt_f2', 'country_f1', 'country_f2', 'age_in_2023_f1',
       'age_in_2023_f2', 'active_f1', 'active_f2', 'Bantamweight',
       'Catch Weight', 'Featherweight', 'Flyweight', 'Heavyweight',
       'Light Heavyweight', 'Lightweight', 'Middleweight', 'Welterweight',
       'Women's Bantamweight', 'Women's Featherweight', 'Women's Flyweight',
       'Women's Strawweight', 'Orthodox_f1', 'Southpaw_f1', 'Switch_f1',
       'Orthodox_f2', 'Southpaw_f2', 'Switch_f2', 'ko', 'dec', 'overturned',
       'sub', 'win_percentage_f1', 'win_percentage_f2', 'sig_str_pm',
       'str_acc_percentage', 'str_abs_pm', 'str_def_percentage', 'td_avg_15m',
       'td_acc_percentage', 'td_def_percentage', '

In [40]:
#you'll see that after the merge, there's a lot of <N/A> and NaN columns. That means our current_fighters database 
#is not complete
#but since this model is for scientific interest and not sports betting, we can use a smaller database
#let's drop the fights that include fighters we don't have additional information on and see how many rows we have left

thresh = 3
merged_fighters.dropna(thresh=merged_fighters.shape[1] - thresh + 1, inplace=True)
merged_fighters.shape

(816, 77)

In [41]:
#816 fights to test is a decent starting point, I think will do just fine

In [42]:
#now let's do a few final touch-ups 
#first, let's create a 'winner' column!

merged_fighters.loc[merged_fighters['w_l'] == 'draw', 'winner'] = 'draw'
merged_fighters.loc[merged_fighters['w_l'] == 'nc nc', 'winner'] = 'nc nc'
merged_fighters.loc[merged_fighters['w_l'] == 'win', 'winner'] = merged_fighters['fighter1']

In [43]:
#unfortunately, the way our data is set up will automatically create a biased model
#because the winner is always going to be fighter1
#to fix that, we're first going to shuffle around half of our rows between fighter1 and fighter2

In [44]:
#let's start by creating a dict of pairs between the columns we'll want to swap

col_pairs = {
    'fighter1': 'fighter2',
    'kd_f1': 'kd_f2',
    'str_f1': 'str_f2',
    'td_f1': 'td_f2',
    'sub_f1': 'sub_f2',
    'ht_f1': 'ht_f2',
    'wt_f1': 'wt_f2',
    'reach_f1': 'reach_f2',
    'w_f1': 'w_f2',
    'l_f1': 'l_f2',
    'd_f1': 'd_f2',
    'belt_f1': 'belt_f2',
    'country_f1': 'country_f2',
    'age_in_2023_f1': 'age_in_2023_f2',
    'active_f1': 'active_f2',
    'Orthodox_f1': 'Orthodox_f2',
    'Southpaw_f1': 'Southpaw_f2',
    'Switch_f1': 'Switch_f2',
    'Orthodox_f2': 'Orthodox_f1',
    'Southpaw_f2': 'Southpaw_f1',
    'Switch_f2': 'Switch_f1',
    'sig_str_pm': 'sig_str_pm_f2',
    'str_acc_percentage': 'str_acc_percentage_f2',
    'str_abs_pm': 'str_abs_pm_f2',
    'str_def_percentage': 'str_def_percentage_f2',
    'td_avg_15m': 'td_avg_15m_f2',
    'td_acc_percentage': 'td_acc_percentage_f2',
    'td_def_percentage': 'td_def_percentage_f2',
    'sub_avg_15m': 'sub_avg_15m_f2',
    'win_percentage_f1': 'win_percentage_f2'
}

In [45]:
#next, we'll randomly pick half of the rows in the dataset
rows_to_swap = len(merged_fighters) // 2
rows_to_swap = np.random.choice(merged_fighters.index, size=rows_to_swap, replace=False)

#and finally, we'll use .loc to switch the keys and values in the dictionary we created for the random selection of rows
for key, value in col_pairs.items():
    merged_fighters.loc[rows_to_swap, [key, value]] = merged_fighters.loc[rows_to_swap, [value, key]].values

In [46]:
#yay! we're all done and ready 
merged_fighters.head()

,w_l,round,time,event,date,location,fighter1,fighter2,kd_f1,kd_f2,str_f1,str_f2,td_f1,td_f2,sub_f1,sub_f2,ht_f1,ht_f2,wt_f1,wt_f2,reach_f1,reach_f2,w_f1,w_f2,l_f1,l_f2,d_f1,d_f2,belt_f1,belt_f2,country_f1,country_f2,age_in_2023_f1,age_in_2023_f2,active_f1,active_f2,Bantamweight,Catch Weight,Featherweight,Flyweight,Heavyweight,Light Heavyweight,Lightweight,Middleweight,Welterweight,Women's Bantamweight,Women's Featherweight,Women's Flyweight,Women's Strawweight,Orthodox_f1,Southpaw_f1,Switch_f1,Orthodox_f2,Southpaw_f2,Switch_f2,ko,dec,overturned,sub,win_percentage_f1,win_percentage_f2,sig_str_pm,str_acc_percentage,str_abs_pm,str_def_percentage,td_avg_15m,td_acc_percentage,td_def_percentage,sub_avg_15m,sig_str_pm_f2,str_acc_percentage_f2,str_abs_pm_f2,str_def_percentage_f2,td_avg_15m_f2,td_acc_percentage_f2,td_def_percentage_f2,sub_avg_15m_f2,winner
1,win,3,5:00,UFC 286: Edwards vs. Usman 3,03/18/23,"London, England, United Kingdom",Rafael Fiziev,Justin Gaethje,0,0,137,145,0,0,0,0,66.0,64.0,125.0,125.0,69.0,64.0,9,21,1,9,0,1,0,0,KGZ,USA,31.0,31.0,TRUE,TRUE,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0.900000,0.677419,5.06,0.51,5.17,0.48,0.31,0.4,0.9,0.0,7.38,0.6,7.66,0.53,0.13,0.25,0.75,0.0,Justin Gaethje
3,win,2,4:27,UFC 286: Edwards vs. Usman 3,03/18/23,"London, England, United Kingdom",Jennifer Maia,Casey O'Neill,0,0,28,10,1,1,1,0,68.0,70.0,135.0,145.0,71.0,72.0,17,17,1,9,0,0,0,0,BRA,CZE,31.0,27.0,TRUE,TRUE,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0.944444,0.653846,4.5,0.38,4.78,0.55,0.27,0.21,0.57,0.1,8.77,0.55,5.9,0.58,1.96,0.42,0.71,0.7,Jennifer Maia
4,win,3,5:00,UFC 286: Edwards vs. Usman 3,03/18/23,"London, England, United Kingdom",Marvin Vettori,Roman Dolidze,0,0,53,47,5,0,0,0,70.0,71.0,155.0,155.0,71.0,73.0,10,11,1,4,0,0,0,0,ITA,GEO,27.0,29.0,FALSE,TRUE,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0.909091,0.733333,4.56,0.45,4.24,0.56,1.66,0.45,0.69,0.5,3.02,0.44,3.01,0.55,1.72,0.5,0.33,1.7,Marvin Vettori
5,win,3,4:32,UFC 286: Edwards vs. Usman 3,03/18/23,"London, England, United Kingdom",Makwan Amirkhani,Jack Shore,0,0,9,16,0,4,2,2,67.0,67.0,125.0,125.0,68.0,7.0,14,10,3,0,0,0,0,0,FIN,WAL,31.0,29.0,FALSE,TRUE,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0.823529,1.000000,1.33,0.44,2.98,0.46,3.96,0.43,0.35,1.1,3.84,0.58,2.2,0.57,3.42,0.4,0.79,0.7,Jack Shore
10,win,5,5:00,UFC 286: Edwards vs. Usman 3,03/11/23,"London, England, United Kingdom",Malcolm Gordon,Jake Hadley,0,0,75,147,1,11,0,0,67.0,66.0,135.0,135.0,67.0,68.0,16,16,5,4,0,0,1,0,CAN,ENG,29.0,27.0,TRUE,TRUE,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0.761905,0.800000,1.93,0.42,2.26,0.42,1.61,0.3,0.09,0.8,4.21,0.48,3.52,0.62,0.47,1.0,0.37,1.9,Jake Hadley


In [47]:
#let's save it to a csv file

merged_fighters.to_csv('merged_fighters.csv')

In [48]:
#that's all the pre-processing we'll do for now! Time for some exploratory analysis!